In [183]:
data = {
    "anchor_url" : 0.2,
    "favicon" : "https://ssl.gstatic.com/colaboratory-static/common/a01fcfa49b15ed5edf590b766bd247d1/img/favicon.ico",
    "iframe" : -1,
    "mail_to" : 1,
    "on_mouse_over" : 1,
    "redirects" : 1,
    "request_url" : 0,
    "server_form_handler" : 1,
    "url" : "https://colab.research.google.com/drive/1DrlPkDg2iEt_rwOLADCNhCe854UjVZ6u#scrollTo=dga-OQmr45VG",
    "origin_url": "https://grabify.link/NV8X47"
}

In [184]:
import urllib
import requests
import whois
from datetime import datetime, timedelta

In [229]:
url_parser = urllib.parse.urlparse(data["url"])
domain = url_parser.netloc
url_path = url_parser.scheme+"://"+domain+url_parser.path

In [231]:
url_parser

ParseResult(scheme='https', netloc='colab.research.google.com', path='/drive/1DrlPkDg2iEt_rwOLADCNhCe854UjVZ6u', params='', query='', fragment='scrollTo=dga-OQmr45VG')

In [186]:
urllib.parse.urlparse(data["url"])

ParseResult(scheme='https', netloc='colab.research.google.com', path='/drive/1DrlPkDg2iEt_rwOLADCNhCe854UjVZ6u', params='', query='', fragment='scrollTo=dga-OQmr45VG')

In [187]:
def contains_IP(url):
    parts = url.split('.')
    if len(parts) == 4 and all(part.isdigit() for part in parts):
        return -1
    else:
        return 1

In [188]:
def url_length(url):
    if(len(url) > 75):
        return -1
    elif(len(url) > 53):
        return 0
    else:
        return 1

In [189]:
def is_shortened(origin_url, url):
    if url != origin_url:
        if(".".join(urllib.parse.urlparse(origin_url).netloc.split(".")[-2:]) == ".".join(urllib.parse.urlparse(url).netloc.split(".")[-2:])):
            return 1
        else:
            return -1
    return 1

In [190]:
def url_contains_at_symbol(url):
    if("@" in url):
        return -1
    else:
        return 1

In [191]:
def url_contains_hyphen(url):
    return -1 if "-" in urllib.parse.urlparse(url).netloc else 1

In [192]:
def sub_domain_count(url):
    count = (urllib.parse.urlparse(url).netloc.count(".")-1)
    if (count>2):
        return -1
    elif (count>1):
        return 0
    else:
        return 1

In [193]:
def contains_https(url):
    return 1 if "https" == urllib.parse.urlparse(url).scheme else -1

In [208]:
def get_domain_age(url):
    whois_ = whois.whois(url)
    creation_date = whois_.creation_date[0] if type(whois_.creation_date)==list else whois_.creation_date
    return 1 if (datetime.now() - creation_date).days>365 else -1

In [207]:
def get_domain_registration_length(url):
    whois_ = whois.whois(url)
    updated_date = whois_.updated_date[0] if type(whois_.updated_date)==list else whois_.updated_date
    expiration_date = whois_.expiration_date[0] if type(whois_.expiration_date)==list else whois_.expiration_date
    return 1 if (expiration_date - updated_date).days>365 else -1

In [180]:
def get_port(url):
    if isinstance((urllib.parse.urlparse(url).port),type(None)):
        return 1
    return -1 if urllib.parse.urlparse(url).port in [443,80,8080] else -1

In [221]:
def is_abnormal_url(url):
    whois_ = whois.whois(url)
    domain_name = whois_.domain_name[0] if type(whois_.domain_name) == list else whois_.domain_name
    return 1 if(domain_name.lower() in url.lower()) else -1

In [225]:
def is_valid_dns_record_present(url):
    try:
        whois.whois(url)
        return 1
    except:
        return -1

In [375]:
def get_host(domain):
    df = pd.read_csv("domain_type.csv")
    chunks = domain.split(".")[::-1]
    terminate = False
    url = ""
    previous_type = None
    for chunk in chunks:
        data = df[df["Domain"] == chunk]
        if(len(data) == 0):
            return chunk+url
        data = data.iloc[0]
        if(data["Type"] == previous_type):
            return chunk+url
        url = "."+chunk+url
        previous_type = data["Type"]
        
    return url

In [389]:
def check_global_ranking(url):
    host = urllib.parse.urlparse(url).netloc
    response = requests.get(f"https://api.similarweb.com/v1/similar-rank/{get_host(host)}/rank?api_key=7965dc2a27124af187e632ced8539bcf")
    if(response.status_code != 200):
        return -1
    response_json = response.json()
    rank = response_json['similar_rank']["rank"]
    if(rank<100000):
        return 1
    else:
        return 0

In [402]:
def check_backlinks(url):
    host = urllib.parse.urlparse(url).netloc
    headers = {
        "Authorization" : "Bearer app#unlogged__6b62b7d688cfc607042ca3a73f13a89103a83fbf"
    }
    response = requests.get(f"https://app.neilpatel.com/api/backlinks_overview?domain={get_host(host)}&mode=domain",headers=headers)
    backlinks = response.json()["backlinks"]
    if(backlinks > 2):
        return 1
    elif(backlinks < 1):
        return -1
    else:
        return 0

In [403]:
check_backlinks(data["url"])

1

In [404]:
check_backlinks("https://sagundev.com.np")

-1

In [390]:
check_global_ranking(data["url"])

1

In [391]:
check_global_ranking("https://pouexam.edu.np")

0

In [281]:
urllib.parse.urlparse("https://pouexam.edu.np")

ParseResult(scheme='https', netloc='pouexam.edu.np', path='', params='', query='', fragment='')

In [282]:
urllib.parse.urlparse(data["url"])

ParseResult(scheme='https', netloc='colab.research.google.com', path='/drive/1DrlPkDg2iEt_rwOLADCNhCe854UjVZ6u', params='', query='', fragment='scrollTo=dga-OQmr45VG')

In [70]:
contains_IP(domain)

1

In [71]:
url_length(data["url"])

-1

In [88]:
is_shortened(data["origin_url"],data["url"])

-1

In [92]:
is_shortened("https://colab.google.com/drive/1DrlPkDg2iEt_rw",data["url"])

1

In [94]:
url_contains_at_symbol("https://abc.xyz.com@")

-1

In [104]:
url_contains_double_slash(data['url'])

1

In [110]:
url_contains_hyphen("https://goog-le.com//facebook.com")

-1

In [125]:
sub_domain_count(data["url"])

0

In [126]:
contains_https(data["url"])

1

In [127]:
contains_https("http://google.com")

-1

In [146]:
get_domain_age(data["url"])

1

In [206]:
get_domain_age("https://scalefluencer.app")

-1

In [182]:
get_port("https://google.com:238")

-1

In [181]:
get_port(data["url"])

1

In [222]:
is_abnormal_url(data["url"])

1

In [226]:
is_valid_dns_record_present(data["url"])

1

In [275]:
whois.whois("pouexam.np")

{'domain_name': None,
 'registrar': None,
 'whois_server': None,
 'referral_url': None,
 'updated_date': None,
 'creation_date': None,
 'expiration_date': None,
 'name_servers': None,
 'status': None,
 'emails': None,
 'dnssec': None,
 'name': None,
 'org': None,
 'address': None,
 'city': None,
 'state': None,
 'registrant_postal_code': None,
 'country': None}